<center><h1>Exploring the Competitors</h1></center>

### Part 1. Get location data using Foursquare

[Foursquare Places API](https://location.foursquare.com/products/places-api/) is very usefule online application used by many developers & other application like Uber etc. In this project you can used it to retrieve informtion about the places present in the neighborhoods of Toronto. The API returns a JSON file and we need to turn that into a data-frame. Here I’ve chosen similar businesses(pet grooming) for each neighborhood within a radius of 2.5km.

You will need to create an account with Foursquare to access the API.  It is free to sign up, and you get $200 free credit.



In [1]:
import requests
import pandas as pd

c:\Users\Martin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Martin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
toronto_DF = pd.read_csv('Toronto_postal_codes_and_coordinates.csv')

In [3]:
display(toronto_DF)

,Unnamed: 0,Borough,Postalcode,Neighborhood,Latitude,Longitude
0,0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160
...,...,...,...,...,...,...
98,98,York,M6C,Humewood-Cedarvale,43.693781,-79.428191
99,99,York,M6E,Caledonia-Fairbanks,43.689026,-79.453512
100,100,York,M6M,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
101,101,York,M6N,"Runnymede, The Junction North",43.673185,-79.487262


In [14]:
# Set up your API key and other constants for Foursquare API
API_KEY = 'fsq3jAlNsDDxUztdezP1wMBdFBKS6ICOZlo2zlRdOu7D1nk='  # Replace with your actual Foursquare API key
LIMIT = 20  # Maximum number of venues to return
radius = 5000  # Search radius in meters

# a function to loop through each neighbourhood in the postalcode.csv file, and search for places
def getNearbyVenues(names, postalcode, latitudes, longitudes, radius):
    venues_list = []

    for name, postalcode, lat, lng in zip(names, postalcode, latitudes, longitudes):
        print(f"Processing neighborhood: {name}")

        # Skip if latitude or longitude is missing
        if pd.isnull(lat) or pd.isnull(lng):
            print(f"Skipping {name} due to missing latitude or longitude.")
            continue

        # Create the API request URL and parameters
        # Change the params here to customize your search, https://docs.foursquare.com/developer/reference/place-search
        # The following example looks for pet groomers in the neighbourhood
        # I am searching with categories ID 11134 - pet grooming services
        url = 'https://api.foursquare.com/v3/places/search'
        params = {
            'query': "pet store",
            'll': f'{lat},{lng}',
            'radius': radius,
            'limit': LIMIT,
        }

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {name}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for venue in results.get('results', []):
            # Extract venue information
            venue_name = venue.get('name')
            venue_id = venue.get('fsq_id')
            venue_location = venue.get('geocodes', {}).get('main', {})
            venue_lat = venue_location.get('latitude')
            venue_lng = venue_location.get('longitude')
            venue_categories = venue.get('categories', [])
            venue_category = venue_categories[0]['name'] if venue_categories else 'Unknown'

            # Append to the list
            venues_list.append([
                name,
                postalcode,
                lat,
                lng,
                venue_name,
                venue_id,
                venue_lat,
                venue_lng,
                venue_category
            ])

    # Create a DataFrame from the list
    nearby_venues = pd.DataFrame(venues_list, columns=[
        'Neighborhood',
        'Postalcode',
        'Neighborhood Latitude',
        'Neighborhood Longitude',
        'Venue',
        'fsq_id',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ])

    return nearby_venues

# Example usage with your DataFrame 'toronto_DF'
# Ensure 'toronto_DF' has the columns 'Neighborhood', 'Latitude', 'Longitude'
toronto_venues = getNearbyVenues(
    names=toronto_DF['Neighborhood'],
    postalcode=toronto_DF['Postalcode'],
    latitudes=toronto_DF['Latitude'],
    longitudes=toronto_DF['Longitude'],
    radius=radius
)

# Display the first few rows of the resulting DataFrame
toronto_venues.head()


Processing neighborhood: Lawrence Park
Processing neighborhood: Davisville North
Processing neighborhood: North Toronto West
Processing neighborhood: Davisville
Processing neighborhood: Moore Park, Summerhill East
Processing neighborhood: Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Processing neighborhood: Roselawn
Processing neighborhood: Forest Hill North & West
Processing neighborhood: The Annex, North Midtown, Yorkville
Processing neighborhood: Rosedale
Processing neighborhood: St. James Town, Cabbagetown
Processing neighborhood: Church and Wellesley
Processing neighborhood: Regent Park, Harbourfront
Processing neighborhood: Garden District, Ryerson
Processing neighborhood: St. James Town
Processing neighborhood: Berczy Park
Processing neighborhood: Central Bay Street
Processing neighborhood: Richmond, Adelaide, King
Processing neighborhood: Harbourfront East, Union Station, Toronto Islands
Processing neighborhood: Toronto Dominion Centre, Design Exchange
Proc

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,M4N,43.72802,-79.38879,PetSmart,580bec0d38fa83192d9e6bb5,43.716614,-79.446053,Pet Supplies Store
1,Lawrence Park,M4N,43.72802,-79.38879,PetSmart,5a0c5ec8fdb9a7258ac7f16d,43.769118,-79.412522,Pet Supplies Store
2,Lawrence Park,M4N,43.72802,-79.38879,Vetster Inc,60a25e64c1e6311b03a170ad,43.730266,-79.402867,Retail
3,Lawrence Park,M4N,43.72802,-79.38879,Bosley's,731468cd36ae7c837f0c58c8,43.732714,-79.403961,Pet Supplies Store
4,Lawrence Park,M4N,43.72802,-79.38879,Pet Valu,4b5b3743f964a520d8eb28e3,43.714116,-79.400148,Pet Supplies Store


In [15]:
# The total number of pet grooming services in Toronto
toronto_venues['Venue'].nunique()

199

In [73]:
# Oberserving the number of competitors in each neighbourhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Agincourt                                          20
Alderwood, Long Branch                             20
Bathurst Manor, Wilson Heights, Downsview North    20
Bayview Village                                    20
Bedford Park, Lawrence Manor East                  20
                                                   ..
Willowdale, Newtonbrook                            20
Woburn                                             20
Woodbine Heights                                   20
York Mills West                                    20
York Mills, Silver Hills                           20
Name: Venue, Length: 103, dtype: int64

In [72]:
toronto_venues.to_excel("toronto_pet_venues.xlsx")

C:\Users\Martin\AppData\Local\Temp/ipykernel_32276/442604555.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  toronto_venues.to_excel("toronto_pet_venues.xlsx")


### Part 2. Interactive leaflet map using coordinate data.


In [17]:
import folium # map rendering library

In [63]:
# My example below shows pet groomers in Lawrence Park
# target = 'Dorset Park, Wexford Heights, Scarborough Town Centre'

# search_area = toronto_venues[toronto_venues['Neighborhood'] == target]
# latitude = toronto_DF[toronto_DF['Neighborhood'] == target]['Latitude']
# longitude = toronto_DF[toronto_DF['Neighborhood'] == target]['Longitude']
# display(search_area)

search_area = toronto_venues
latitude = toronto_DF['Latitude'].mean()
longitude = toronto_DF['Longitude'].mean()
display(toronto_venues)

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,M4N,43.728020,-79.388790,PetSmart,580bec0d38fa83192d9e6bb5,43.716614,-79.446053,Pet Supplies Store
1,Lawrence Park,M4N,43.728020,-79.388790,PetSmart,5a0c5ec8fdb9a7258ac7f16d,43.769118,-79.412522,Pet Supplies Store
2,Lawrence Park,M4N,43.728020,-79.388790,Vetster Inc,60a25e64c1e6311b03a170ad,43.730266,-79.402867,Retail
3,Lawrence Park,M4N,43.728020,-79.388790,Bosley's,731468cd36ae7c837f0c58c8,43.732714,-79.403961,Pet Supplies Store
4,Lawrence Park,M4N,43.728020,-79.388790,Pet Valu,4b5b3743f964a520d8eb28e3,43.714116,-79.400148,Pet Supplies Store
...,...,...,...,...,...,...,...,...,...
2029,Weston,M9N,43.706876,-79.518188,Pet Valu,4d3997f66de7721e8719e649,43.679600,-79.546915,Pet Supplies Store
2030,Weston,M9N,43.706876,-79.518188,Bosley's,8931c2a4ae2dd1e4e25d749e,43.728394,-79.480073,Pet Supplies Store
2031,Weston,M9N,43.706876,-79.518188,Pet Valu,4d95ce679079b1f7dc7ce909,43.728342,-79.480018,Pet Supplies Store
2032,Weston,M9N,43.706876,-79.518188,Paulmac's,f17ac15f94552387804be226,43.728394,-79.480073,Pet Supplies Store


In [61]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area['Venue Latitude'], search_area['Venue Longitude'], search_area['Venue'], search_area['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [68]:
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import numpy as np
import io
from PIL import Image
import base64

# Sample function to generate a pie chart as an image
def generate_pie_chart(venue_counts):
    labels = venue_counts.keys()
    sizes = venue_counts.values()

    # Create a figure and axis for the pie chart
    fig, ax = plt.subplots(figsize=(3, 3))
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

    # Save the pie chart to a buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    img = Image.open(buf)
    
    # Convert to base64 for embedding in folium popup
    img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    return img_base64

# Define a function to add venues to the map
def create_map_with_pie_charts(venues_df):
    # Create an empty map centered around Toronto (you can change this to a specific center)
    map_center = [43.7, -79.42]  # Toronto's approximate center
    m = folium.Map(location=map_center, zoom_start=12)

    # Initialize MarkerCluster for better performance with many markers
    marker_cluster = MarkerCluster().add_to(m)

    # Group venues by neighborhood and count them
    neighborhood_groups = venues_df.groupby('Neighborhood')

    for neighborhood, group in neighborhood_groups:
        # Count venues in the neighborhood
        venue_counts = group['Venue Category'].value_counts()

        # Generate the pie chart image
        pie_chart_img = generate_pie_chart(venue_counts)

        # Compute the number of venues in the neighborhood for dot size
        num_venues = len(group)

        # Set the dot size based on the number of venues
        dot_size = num_venues * 2  # You can adjust the multiplier as needed

        # Get the latitude and longitude of the neighborhood (take the mean)
        lat = group['Venue Latitude'].mean()
        lon = group['Venue Longitude'].mean()

        # Create a popup with the pie chart image
        popup_html = f'<img src="data:image/png;base64,{pie_chart_img}" width="150" height="150">'

        # Create a circle marker for the neighborhood
        folium.CircleMarker(
            location=[lat, lon],
            radius=dot_size,  # Size based on the number of venues
            color='blue',
            fill=True,
            fill_opacity=0.6,
            popup=folium.Popup(popup_html, max_width=200)
        ).add_to(marker_cluster)

    # Return the map
    return m

# Example usage with your DataFrame 'venues_df' (make sure it contains 'Venue Latitude', 'Venue Longitude', 'Neighborhood', 'Venue Category')
# Assuming venues_df is already created from previous steps
map_with_pie_charts = create_map_with_pie_charts(search_area)

# Display the map
map_with_pie_charts

TypeError: 'numpy.ndarray' object is not callable

### Part 3. Customer Reviews
It seems like there is one one pet groomer withing 2km of Scarborough Village, I am interested in seeing customer's comments on Funny Bunny.  I can use [Foursquare's Place Tips API.](https://docs.foursquare.com/developer/reference/place-tips)


In [ ]:
# a function to loop through the list of pet groomers in the neighborhood and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"

        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }
        
        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()
        
        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')
   

            # Append to the list
            venues_tips.append([
                venues,
                tip_id,
                tip_created,
                tip_text
            ])
            
    
    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])

    return nearby_tips


search_sample = search_area.sample(n=1000, replace=False)

nearby_tips = getTips(
    venues=search_sample['Venue'],
    fsq_ids=search_sample['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: All Reptiles
Processing venue: Paulmac's
Processing venue: Reefquarium & Marine Ltd
Processing venue: Paulmac's
Processing venue: Sit Stay Dog Emporium
Processing venue: The Bone House
Processing venue: Bosley's
Processing venue: Global Pet Foods
Processing venue: Ren's Pets
Processing venue: Bosley's
Processing venue: Paulmac's
Processing venue: Wongs Aquarium
Processing venue: Ren's Pets
Processing venue: Paulmac's
Processing venue: Pet Valu
Processing venue: Helmutt's Pet Supply
Processing venue: Pet Valu
Processing venue: Pet Valu
Processing venue: PetSmart
Processing venue: Small Wonders Pet Emporium Inc
Processing venue: Mind Your Manners
Processing venue: Global Pet Foods
Processing venue: MyPawStar
Processing venue: Captain K9
Processing venue: Bosley's
Processing venue: Gentle Paw
Processing venue: Furry Friends Training
Processing venue: The Bone & Biscuit Co
Processing venue: Endless Tails Pet Nutrition Centre
Processing venue: Paulmac's
Processing venue: G

,venue_name,id,created_at,text
0,1725 All Reptiles 659 ...,4ed6901b77c88f53d7c79e52,2011-11-30T20:20:43.000Z,Awesome place! Great selection and very knowle...
1,1725 All Reptiles 659 ...,4fd363c5e4b0f0b2d4a0d7ce,2012-06-09T14:55:01.000Z,great people.
2,1725 All Reptiles 659 ...,4fd5141ee4b0ba0232d72527,2012-06-10T21:39:42.000Z,"Try the doggy gelato, a definite hit."
3,1725 All Reptiles 659 ...,528b9c16498e272d66692de0,2013-11-19T17:12:54.000Z,Great products that keep your doggie healthy a...
4,1725 All Reptiles 659 ...,4c46078f342c1b8d0b5cb589,2010-07-20T20:31:11.000Z,Go in and pet Igor. He loves the attention.


### Now it is your turn to discover interesting search results using Foursquare
You are expected to read through the API documentation and customize the searches.  